In [2]:
# Standard libraries
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interactive, IntProgress
from IPython.display import display

# Append base directory
import os,sys,inspect
rootname = "neuronal-sequence-test"
thispath = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
rootpath = os.path.join(thispath[:thispath.index(rootname)], rootname)
sys.path.append(rootpath)
print("Appended root directory", rootpath)

# User libraries
from src.lib.qt_wrapper import gui_fnames, gui_fpath
from src.lib.data_db import BehaviouralNeuronalDatabase

%load_ext autoreload
%autoreload 2

Appended root directory /home/alyosha/work/git/neuronal-sequence-test


# TODO

1. Preprocessing : Apply background subtraction
2. Alignment : Ensure intervals actually match. Ask Maria for visual test of alignment
3. Apply metrics - 1 data point per trial - pool trials - maintenance vs all
    * 1D semistatic: Mean, Variance, 1D Entropy
    * ND semistatic: Avg Correlation, ND Entropy
    * 1D dynamic: Predictive Info, AR(1) efficiency
    * ND dynamic: Predictive Info, AR(1) efficiency, Orderability coeff
    
4. Test Hypotheses:
    1. Maintenance predicts Correct/Incorrect vs other phases
    2. Maintenance predicts L/R better vs other phases
    3. Validation: Prev trial C/I or L/R predicts that of next trial
    4. Predictive aspect:
        * Semi-static maintenance activity is good predictor, fine temporal details irrelevant
        * Semi-static maintenance activity is not good, fine temporal details improve prediction

In [3]:
# tmp_path = root_path_data if 'root_path_data' in locals() else "./"
params = {}
params['root_path_data']  = gui_fpath("Path to data files", "./")

In [4]:
dataDB = BehaviouralNeuronalDatabase(params)

In [5]:
dataDB.read_neuro_files()

IntProgress(value=0, description='Read Neuro Data:', max=4)

In [6]:
dataDB.metaDataFrames['behavior']

,mousename,date,mousekey,path
0,m060,2019-05-22,m060_20190522,/media/alyosha/Data/TE_data/mariadata/m060/Beh...
1,m060,2019-05-24,m060_20190524,/media/alyosha/Data/TE_data/mariadata/m060/Beh...
2,m060,2019-05-27,m060_20190527,/media/alyosha/Data/TE_data/mariadata/m060/Beh...
3,m060,2019-06-06,m060_20190606,/media/alyosha/Data/TE_data/mariadata/m060/Beh...


In [7]:
dataDB.read_behavior_files()

IntProgress(value=0, description='Read Neuro Data:', max=4)

Reading /media/alyosha/Data/TE_data/mariadata/m060/Behavior_m060_20190522.mat
Reading /media/alyosha/Data/TE_data/mariadata/m060/Behavior_m060_20190524.mat
Reading /media/alyosha/Data/TE_data/mariadata/m060/Behavior_m060_20190527.mat
Reading /media/alyosha/Data/TE_data/mariadata/m060/Behavior_m060_20190606.mat
No trials found for Trial_LWhole_Mistake skipping


In [20]:
statLst = dataDB.get_phase_all("Maintenance", "L", "Correct")

[(699, 770), (990, 1123), (1952, 2029), (3312, 3374), (3977, 4045), (4894, 4959), (5248, 5312), (6066, 6159), (6666, 6736), (8203, 8321), (9239, 9299), (9531, 9589), (9806, 9865), (10448, 10520), (11354, 11416), (12560, 12627), (12836, 12902), (13437, 13532), (14030, 14098), (15217, 15285), (15876, 15956)]
[(446, 509), (1325, 1439), (1754, 1822), (2407, 2485), (3236, 3300), (3571, 3634), (4632, 4698), (4963, 5048), (6847, 6915), (7208, 7272), (7932, 8007), (8875, 8937), (9488, 9553), (10478, 10637), (10989, 11052), (11276, 11346), (11958, 12070), (12789, 12861), (13658, 13749)]
[(651, 743), (1037, 1115), (2802, 2920), (3545, 3608), (4223, 4286), (5321, 5386), (5992, 6056), (6682, 6738), (7154, 7215), (7513, 7581), (8435, 8512), (9080, 9161), (9403, 9585), (9798, 9856), (10361, 10428), (11339, 11411)]
[(2143, 2265), (2691, 2830), (3634, 3704), (4015, 4102), (4438, 4532), (5235, 5331), (5625, 5712), (6441, 6536), (7282, 7373), (7806, 7940), (8223, 8334), (9518, 9623), (9926, 10025), (113

In [22]:
[st.shape for st in statLst]

[(71, 88),
 (133, 88),
 (77, 88),
 (62, 88),
 (68, 88),
 (65, 88),
 (64, 88),
 (93, 88),
 (70, 88),
 (118, 88),
 (60, 88),
 (58, 88),
 (59, 88),
 (72, 88),
 (62, 88),
 (67, 88),
 (66, 88),
 (95, 88),
 (68, 88),
 (68, 88),
 (80, 88),
 (63, 88),
 (114, 88),
 (68, 88),
 (78, 88),
 (64, 88),
 (63, 88),
 (66, 88),
 (85, 88),
 (68, 88),
 (64, 88),
 (75, 88),
 (62, 88),
 (65, 88),
 (159, 88),
 (63, 88),
 (70, 88),
 (112, 88),
 (72, 88),
 (91, 88),
 (92, 88),
 (78, 88),
 (118, 88),
 (63, 88),
 (63, 88),
 (65, 88),
 (64, 88),
 (56, 88),
 (61, 88),
 (68, 88),
 (77, 88),
 (81, 88),
 (182, 88),
 (58, 88),
 (67, 88),
 (72, 88),
 (122, 88),
 (139, 88),
 (70, 88),
 (87, 88),
 (94, 88),
 (96, 88),
 (87, 88),
 (95, 88),
 (91, 88),
 (134, 88),
 (111, 88),
 (105, 88),
 (99, 88),
 (104, 88),
 (98, 88),
 (126, 88),
 (118, 88),
 (102, 88)]